In [5]:
# %% [markdown]
# # Fase 2: Refinar e Explodir Colunas de Práticas e Degradação
# Mantém só colunas específicas, expande múltiplas respostas e gera contagens por município.

import pandas as pd
import os

# Caminhos
input_dir = "C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Normalizados_Munic"
output_dir = "C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Explodidos"
os.makedirs(output_dir, exist_ok=True)

# Dicionário para controlar as colunas que você quer manter em cada arquivo
# (deixe só as que precisa e ajuste conforme for revisando)
arquivos_colunas = {
    "01_Atividades_Mun.csv": ["atividade_produtiva","codigo_final", "municipio_ibge", "Record Count"],
    "01_Politicas_Publicas_Mun.csv": ["acessa_ou_ja_acessou_politicas_publicas__","codigo_final","municipio_ibge", "Record Count"],
    "01_Uso_Convencionais_Degradacao_Mun.csv": [
        "uso_de_praticas_convencionais",
        "qual_o_nivel_de_degradacao_que_predomina_nas_pastagens_",
        "municipio_ibge",
        "Record Count"
    ],
    "01_Uso_Sustentavel_Degradacao_Mun.csv": [
        "uso_de_praticas_sustentaveis_",
        "qual_o_nivel_de_degradacao_que_predomina_nas_pastagens_",
        "municipio_ibge",
        "Record Count"
    ]
}

# %% Função auxiliar para explodir colunas com múltiplas respostas
def expandir_colunas_multiplas(df, colunas_multiplas):
    """Divide múltiplas respostas em linhas separadas."""
    for col in colunas_multiplas:
        if col in df.columns:
            # divide por vírgula, ponto e vírgula, barra, ' e ' etc.
            df[col] = df[col].astype(str).str.replace(" e ",",").str.replace("/", ",")
            df = df.assign(**{col: df[col].str.split(",")}).explode(col)
            df[col] = df[col].str.strip().str.capitalize()
    return df

# %% Processar cada arquivo
for nome_arq, colunas in arquivos_colunas.items():
    caminho_in = os.path.join(input_dir, nome_arq)
    if not os.path.exists(caminho_in):
        print(f"⚠️ Arquivo não encontrado: {nome_arq}")
        continue

    print(f"🔧 Processando {nome_arq} ...")
    df = pd.read_csv(caminho_in)

    # Selecionar apenas colunas existentes (ignora se alguma faltar)
    colunas_existentes = [c for c in colunas if c in df.columns]
    df = df[colunas_existentes].copy()

    # Renomear 'Record Count' se quiser deixar mais simples
    df = df.rename(columns={"Record Count": "count"})

    # Detectar colunas com múltiplas respostas (heurística simples)
    colunas_multiplas = [c for c in df.columns if df[c].astype(str).str.contains(",|/| e ").any()]

    # Explodir colunas múltiplas
    df_exp = expandir_colunas_multiplas(df, colunas_multiplas)

    # Agrupar por município e coluna específica (se fizer sentido)
    if len(colunas_multiplas) > 0:
        for col in colunas_multiplas:
            resumo = (
                df_exp.groupby(["municipio_ibge", col])
                .agg(total_respostas=("count", "sum"))
                .reset_index()
            )
            nome_base = nome_arq.replace(".csv", f"_{col}_contagem.csv")
            caminho_out = os.path.join(output_dir, nome_base)
            resumo.to_csv(caminho_out, index=False, encoding="utf-8-sig")
            print(f"✅ Exportado: {caminho_out}")
    else:
        caminho_out = os.path.join(output_dir, nome_arq)
        df_exp.to_csv(caminho_out, index=False, encoding="utf-8-sig")
        print(f"✅ Exportado (sem múltiplas colunas): {caminho_out}")

print("\n✨ Processamento concluído!")


🔧 Processando 01_Atividades_Mun.csv ...
✅ Exportado: C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Explodidos\01_Atividades_Mun_atividade_produtiva_contagem.csv
🔧 Processando 01_Politicas_Publicas_Mun.csv ...
✅ Exportado: C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Explodidos\01_Politicas_Publicas_Mun_acessa_ou_ja_acessou_politicas_publicas___contagem.csv
🔧 Processando 01_Uso_Convencionais_Degradacao_Mun.csv ...
✅ Exportado: C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Explodidos\01_Uso_Convencionais_Degradacao_Mun_uso_de_praticas_convencionais_contagem.csv
🔧 Processando 01_Uso_Sustentavel_Degradacao_Mun.csv ...
✅ Exportado: C:/Users/Valentine/Artigo_Mapitos/data_clean/Dados_Explodidos\01_Uso_Sustentavel_Degradacao_Mun_uso_de_praticas_sustentaveis__contagem.csv

✨ Processamento concluído!
